# Iteración de Valores

En este ejercicio vamos a implementar el primer método para solucionar Procesos de Decisión de Markov (MDPs). El método a implementar es la iteración de valores.

La iteración de valores esta basada en la fórmula:

![value_iteration](./img/value_iteration.png)

Para resolver los MDPs utilizaremos `value_iteration.py` para la lógica de la iteración de valores y `gridworld_mdp.py` para definir los ambientes de Gridworld y Bridge.


In [2]:
from value_iteration import ValueIteration
from gridworld_mdp import GridworldMDP, BridgeMDP
import numpy as np

def print_grid_values(mdp, values, title="Valores de Estados"):
    print(f"\n--- {title} ---")
    grid_display = []
    for r in range(mdp.nrows):
        row_display = []
        for c in range(mdp.ncols):
            state = (r, c)
            if mdp.board[r][c] == '#':
                row_display.append("  #  ")
            elif mdp.is_terminal(state):
                row_display.append(f"{float(mdp.board[r][c]):+.2f}")
            else:
                row_display.append(f"{values.get(state, 0.0):+.2f}")
        grid_display.append(" ".join(row_display))
    for row in grid_display:
        print(row)

def print_grid_policy(mdp, policy, title="Política de Acciones"):
    print(f"\n--- {title} ---")
    action_map = {'up': '^^', 'down': 'VV', 'left': '<<', 'right': '>>', 'exit': 'EX', None: '--'}
    grid_display = []
    for r in range(mdp.nrows):
        row_display = []
        for c in range(mdp.ncols):
            state = (r, c)
            if mdp.board[r][c] == '#':
                row_display.append(" # ")
            elif mdp.is_terminal(state):
                row_display.append(f" {mdp.board[r][c]:2s} ") # Display reward for terminal states
            else:
                action = policy.get(state, None)
                row_display.append(action_map.get(action, '?') + " ")
        grid_display.append(" ".join(row_display))
    for row in grid_display:
        print(row)


## Gridworld: Configuración y Experimentos

Utilizaremos un tablero de 10x10 para el ambiente de Gridworld, con el modelo de transición de probabilidad uniforme (0.25 para cada acción) como se planteó en la Task 2 de `Assignment_gridworld.ipynb`.


In [3]:
board_10x10 = [[' ' for _ in range(10)] for _ in range(10)]
board_10x10[0][0] = 'S'
for c in [1, 2, 3, 4, 6, 7, 8]: board_10x10[2][c] = '#'
for r in [3, 4, 5, 6, 7]: board_10x10[r][4] = '#'
board_10x10[4][5] = '-1' # R: -1
board_10x10[5][5] = '+1' # R: 1
board_10x10[7][5] = '-1' # R: -1
board_10x10[7][6] = '-1' # R: -1

gridworld_mdp = GridworldMDP(board_10x10, transition_model="uniform_0.25")

# PEGA ESTO EN SU LUGAR:
iterations_to_test = [5, 10, 15, 20, 30, 50]

# Variables para guardar el estado de la iteración anterior
previous_values = {}
previous_policy = {}

print("\n===== Resumen de Convergencia =====")
print(f"{'Iteraciones':<15} | {'Max Delta (Δ)':<15} | {'¿Cambió la Política?':<20}")
print("-" * 55)

for num_iterations in iterations_to_test:
    vi_agent = ValueIteration(gridworld_mdp, discount=0.9, iterations=num_iterations)
    vi_agent.run_value_iteration()
    
    current_values = vi_agent.values
    current_policy = {state: vi_agent.get_policy(state) for state in gridworld_mdp.get_states()}
    
    if previous_values:
        max_delta = max(abs(current_values.get(s, 0.0) - previous_values.get(s, 0.0)) for s in gridworld_mdp.get_states())
    else:
        max_delta = max(abs(current_values.get(s, 0.0)) for s in gridworld_mdp.get_states()) 
        
    if previous_policy:
        policy_changed = current_policy != previous_policy
        cambio_str = "Sí" if policy_changed else "No"
    else:
        cambio_str = "Base (Inicial)"
        
    print(f"{num_iterations:<15} | {max_delta:<15.4f} | {cambio_str:<20}")
    
    previous_values = current_values.copy()
    previous_policy = current_policy.copy()

print("-" * 55)
print("Nota: El 'Max Delta (Δ)' muestra cuánto cambiaron los valores respecto a la iteración probada anterior.")


===== Resumen de Convergencia =====
Iteraciones     | Max Delta (Δ)   | ¿Cambió la Política?
-------------------------------------------------------
5               | 0.4043          | Base (Inicial)      
10              | 0.0708          | No                  
15              | 0.0264          | No                  
20              | 0.0134          | No                  
30              | 0.0101          | No                  
50              | 0.0031          | No                  
-------------------------------------------------------
Nota: El 'Max Delta (Δ)' muestra cuánto cambiaron los valores respecto a la iteración probada anterior.


A partir de los resultados, observamos una propiedad empírica clave del algoritmo de Iteración de Valores: la política óptima converge mucho más rápido que los valores de los estados. Mientras que la función de valor continúa refinándose numéricamente hacia un estado estacionario —evidenciado por la reducción constante del cambio máximo (Max Delta) de 0.4043 en las primeras 5 iteraciones a apenas 0.0031 en la iteración 50—, las acciones óptimas a tomar no sufren ninguna alteración desde la iteración 5. Esta estabilización temprana demuestra que el algoritmo encuentra el orden relativo correcto de las acciones mucho antes de calcular el valor numérico exacto de cada estado, garantizando que el agente esté listo para tomar decisiones óptimas en el MDP desde las etapas iniciales del proceso.


## Bridge Environment: Configuración y Experimentos

El ambiente del puente se define como una matriz de `3x7`:

- Filas 0 y 2: Tienen recompensa -100 entre las columnas 2 y 5 (inclusive). Las demás celdas son estados normales (recompensa 0).
- Fila 1: Es el puente. La entrada es en `(1,0)` (estado inicial 'S') y la salida en `(1,6)` con recompensa +100.

Analizaremos el comportamiento con factores de descuento de 0.9 y 0.1.


In [4]:
bridge_board = [
    [' ', ' ', '-100', '-100', '-100', '-100', ' '],
    ['S', ' ', ' ',    ' ',    ' ',    ' ',    '+100'],
    [' ', ' ', '-100', '-100', '-100', '-100', ' ']
]

bridge_mdp = BridgeMDP(bridge_board, transition_model="deterministic")

print("\n===== Bridge Environment (Discount = 0.9) =====")
vi_agent_09 = ValueIteration(bridge_mdp, discount=0.9, iterations=10)
vi_agent_09.run_value_iteration()
policy_09 = {state: vi_agent_09.get_policy(state) for state in bridge_mdp.get_states()}
print_grid_values(bridge_mdp, vi_agent_09.values, "Valores de Estados (Discount=0.9)")
print_grid_policy(bridge_mdp, policy_09, "Política Optima (Discount=0.9)")

print("\n===== Bridge Environment (Discount = 0.1) =====")
vi_agent_01 = ValueIteration(bridge_mdp, discount=0.1, iterations=10)
vi_agent_01.run_value_iteration()
policy_01 = {state: vi_agent_01.get_policy(state) for state in bridge_mdp.get_states()}
print_grid_values(bridge_mdp, vi_agent_01.values, "Valores de Estados (Discount=0.1)")
print_grid_policy(bridge_mdp, policy_01, "Política Optima (Discount=0.1)")



===== Bridge Environment (Discount = 0.9) =====

--- Valores de Estados (Discount=0.9) ---
+53.14 +59.05 -100.00 -100.00 -100.00 -100.00 +100.00
+59.05 +65.61 +72.90 +81.00 +90.00 +100.00 +100.00
+53.14 +59.05 -100.00 -100.00 -100.00 -100.00 +100.00

--- Política Optima (Discount=0.9) ---
VV  VV   -100   -100   -100   -100  VV 
>>  >>  >>  >>  >>  >>   +100 
^^  ^^   -100   -100   -100   -100  ^^ 

===== Bridge Environment (Discount = 0.1) =====

--- Valores de Estados (Discount=0.1) ---
+0.00 +0.00 -100.00 -100.00 -100.00 -100.00 +100.00
+0.00 +0.01 +0.10 +1.00 +10.00 +100.00 +100.00
+0.00 +0.00 -100.00 -100.00 -100.00 -100.00 +100.00

--- Política Optima (Discount=0.1) ---
VV  VV   -100   -100   -100   -100  VV 
>>  >>  >>  >>  >>  >>   +100 
^^  ^^   -100   -100   -100   -100  ^^ 


### Análisis del Factor de Descuento en Bridge

- Con un **descuento alto (e.g., 0.9)**, el agente considera las recompensas futuras casi tan valiosas como las inmediatas. Esto lo incentivará a buscar la recompensa de +100 en el puente, incluso si eso significa pasar por más pasos para llegar allí, evitando las recompensas negativas de -100.

- Con un **descuento bajo (e.g., 0.1)**, el agente prioriza las recompensas inmediatas. Las recompensas futuras se devalúan rápidamente. En este caso, el agente podría ser más propenso a tomar caminos más cortos, incluso si esto conlleva el riesgo de encontrarse con una recompensa negativa en el futuro, ya que el impacto de esa recompensa negativa futura es mucho menor. Podría optar por evitar el puente si un camino más corto lo lleva a una recompensa (o penalización) que no está demasiado lejos, o podría simplemente quedarse quieto si no ve una recompensa significativa a corto plazo.
